In [ ]:
from utils import data, comunity_detection, mf_dcv, ic, fitness
import numpy as np
import random
import math

In [9]:
import networkx as nx

In [1]:
K_SEEDS = 40
POP_SIZE = 10
MAX_GEN = 150
P_CROSSOVER = 0.6
P_MUTATION = 0.1
LAMBDA_VAL = 0.5
PROPAGATION_PROB = 0.01
MC_SIMULATIONS = 1000

In [31]:
def evuluate(individual, G, groups, ideal_influences, cache):
    mf, dcv = mf_dcv.calculate_MF_DCV(
        G, individual, groups, ideal_influences,
        p=PROPAGATION_PROB, mc=MC_SIMULATIONS, cache=cache
    )
    fit = fitness.fitness_F(mf, dcv, LAMBDA_VAL)
    return individual, mf, dcv, fit

In [4]:
G, node_groups_map = data.load_data_from_pickle("dataset/rice_subset.pickle", "color")

In [5]:
groups = {}
for n, g in node_groups_map.items():
    if g not in groups: groups[g] = []
    groups[g].append(n)

In [20]:
ideal_influences = {}
N = len(G)
for g_id, nodes in groups.items():
    k_i = math.ceil(K_SEEDS * len(nodes) / N)
    subgraph = G.subgraph(nodes)
    ideal = ic.greedy_max_influence(subgraph, k_i, p=PROPAGATION_PROB, mc=30)
    ideal_influences[g_id] = ideal

In [21]:
nx.pagerank(G)

{3073: 0.0013220402156670437,
 4099: 0.0016966718705756477,
 5: 0.003407970960317275,
 3078: 0.003195985110533711,
 2055: 0.00137711094469001,
 6152: 0.002408670408439683,
 11: 0.002177859950801878,
 12: 0.0022925393887874798,
 13: 0.0015913985059070842,
 3087: 0.0016091237850712767,
 3088: 0.0016157356685005846,
 3090: 0.002206492252521324,
 20: 0.0032266639052355357,
 1048: 0.0035525140701528806,
 1049: 0.002455785754379061,
 3099: 0.0018820559611105049,
 2053: 0.0013940437116910177,
 2073: 0.0025454117260399374,
 2089: 0.0020148626494057974,
 3116: 0.0013293541377420747,
 5165: 0.0028877184451241073,
 5168: 0.0015749134729233225,
 1074: 0.0018775943986277892,
 3123: 0.0016378685821399903,
 350: 0.0012986112730498556,
 54: 0.002813017891242294,
 56: 0.001880083040340221,
 57: 0.000742941717880079,
 2866: 0.0010777330580483432,
 63: 0.002289243559024877,
 2113: 0.002910919496075545,
 1092: 0.002483922506903546,
 5473: 0.0028906163208334253,
 72: 0.0029397205530876215,
 3145: 0.0024943

In [22]:
SN_scores = nx.pagerank(G)

In [23]:
communities, _ = comunity_detection.get_community_structure(G)

In [24]:
A_j_counts = {g: len(nodes) for g, nodes in groups.items()}

In [25]:
SC_scores = comunity_detection.calculate_SC(communities, G, None, A_j_counts)

In [ ]:
population = []
comm_ids = list(communities.keys())
comm_scores_vec = np.array([SC_scores.get(cid, 0) for cid in comm_ids], dtype=float)
comm_scores_vec = np.clip(comm_scores_vec, 0.0, None)

for _ in range(POP_SIZE):
    ind = comunity_detection.community_based_selection(G, K_SEEDS, communities, SN_scores, SC_scores)
    population.append(ind)

    # Vectorized community allocation by probability
    probs = comm_scores_vec.copy()
    s = probs.sum()
    if s == 0:
        probs = np.ones_like(probs) / len(probs)
    else:
        probs = probs / s
    picks = np.random.choice(len(comm_ids), size=K_SEEDS, replace=True, p=probs)
    counts = {}
    for idx in picks:
        cid = comm_ids[idx]
        counts[cid] = counts.get(cid, 0) + 1

    # Select top-SN per picked community
    fairness_random_solution = []
    for cid, cnt in counts.items():
        if cnt <= 0:
            continue
        nodes_in_comm = list(communities[cid])
        if not nodes_in_comm:
            continue
        sn_vals = np.array([SN_scores.get(x, 0.0) for x in nodes_in_comm])
        order = np.argsort(-sn_vals)
        selected_nodes = [nodes_in_comm[i] for i in order[:cnt]]
        fairness_random_solution.extend(selected_nodes)

    # Fill up remaining by global top-SN excluding already chosen
    if len(fairness_random_solution) < K_SEEDS:
        all_nodes = list(G.nodes())
        sn_all = np.array([SN_scores.get(x, 0.0) for x in all_nodes])
        order_all = np.argsort(-sn_all)
        chosen = set(fairness_random_solution)
        for i in order_all:
            n = all_nodes[i]
            if n not in chosen:
                fairness_random_solution.append(n)
                chosen.add(n)
            if len(fairness_random_solution) >= K_SEEDS:
                break
    fairness_random_solution = fairness_random_solution[:K_SEEDS]
    population.append(fairness_random_solution)

    # SN-weighted random solution (vectorized)
    all_nodes = list(G.nodes())
    if all_nodes:
        weights = np.array([SN_scores.get(n, 0.0) + 1e-8 for n in all_nodes], dtype=float)
        sw = weights.sum()
        if sw == 0:
            k_pick = min(K_SEEDS, len(all_nodes))
            random_weighted_solution = random.sample(all_nodes, k_pick)
        else:
            probs_nodes = weights / sw
            k_pick = min(K_SEEDS, len(all_nodes))
            random_weighted_solution = list(np.random.choice(all_nodes, size=k_pick, replace=False, p=probs_nodes))
        population.append(random_weighted_solution)

In [33]:
best_S = None
best_Fit = -999
best_metrics = (0, 0)

for gen in range(MAX_GEN):
    influence_cache = {}
    # Evaluate sequentially
    results = []
    for ind in population:
        result = evuluate(ind, G, groups, ideal_influences, influence_cache)
        results.append(result)

    fitnesses = []
    for ind, mf, dcv, fit in results:
        fitnesses.append(fit)

        if fit > best_Fit:
            best_Fit = fit
            best_S = ind
            best_metrics = (mf, dcv)
    
    # Selection
    sorted_idx = np.argsort(fitnesses)[::-1]
    population = [population[i] for i in sorted_idx[:POP_SIZE]]

    # Crossover & Mutation
    new_pop = []
    new_pop.extend(population[:2]) # Elitism
    
    while len(new_pop) < POP_SIZE:
        idx1 = np.random.randint(0, len(population))
        idx2 = np.random.randint(0, len(population))
        p1 = population[idx1]
        p2 = population[idx2]
        
        # Crossover
        if np.random.random() < P_CROSSOVER:
            combined = list(set(p1) | set(p2))
            # Sort theo SN score để lấy top node
            combined.sort(key=lambda x: SN_scores.get(x, 0), reverse=True)
            child = combined[:K_SEEDS]
        else:
            child = p1[:]
        
        # Mutation
        if np.random.random() < P_MUTATION and len(child) > 0:
            idx_remove = np.random.randint(0, len(child))
            child.pop(idx_remove)
            
            comm_keys = list(communities.keys())
            if comm_keys:
                comm_id = np.random.choice(comm_keys)
                candidates = communities[comm_id]
                if candidates:
                    cand = np.random.choice(candidates)
                    if cand not in child: child.append(cand)
                    elif len(p1) > idx_remove: child.append(p1[idx_remove])
        
        # Fill if missing
        while len(child) < K_SEEDS:
            possible = list(set(G.nodes()) - set(child))
            if not possible: break
            child.append(np.random.choice(possible))
            
        
        # Trim if excess
        child = child[:K_SEEDS]
        
        new_pop.append(child)
        
    population = new_pop
    print(f"Gen {gen+1}: Best Fit={best_Fit:.4f} | MF={best_metrics[0]:.4f}, DCV={best_metrics[1]:.4f}")

Gen 1: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 2: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 3: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 4: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 5: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 6: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 7: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 8: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 9: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 10: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 11: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 12: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 13: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 14: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 15: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 16: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 17: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 18: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 19: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 20: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
Gen 21: Best Fit=0.0560 | MF=0.1686, DCV=0.0566
G